In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pointbiserialr
import numpy as np

In [ ]:
del_columns = []
pd.set_option('display.float_format', '{:.4f}'.format)

url = "https://www.dropbox.com/scl/fi/uvv7j1bragzqkz9zwyvj0/sample_mmp.csv?rlkey=i0mlaxzq6e3blblfu9mhrdpsm&e=1&dl=1"
df = pd.read_csv(url)

df

In [3]:
fran_columns = list(df.columns[0:21])
ignacio_columns = list(df.columns[21:42])
marc_columns = list(df.columns[42:63])
alvaro_columns = list(df.columns[63:85])

In [ ]:
df.info()

In [ ]:
df.describe().T

## Parte JUANFRAN

### "Unnamed: 0"

In [ ]:
df["Unnamed: 0"].value_counts()

In [ ]:
df["Unnamed: 0"].nunique()

Podemos ver que la columna "Unnamed: 0" contiene valores unicos que no se repiten, por lo que podemos suponer que a lo mejor puede ser algun tipo de indice interno que tiene la BBDD, por lo que prescindiremos de esta columna

In [8]:
del_columns.append("Unnamed: 0")

### "MachineIdentifier"

In [ ]:
df["MachineIdentifier"].value_counts()

In [ ]:
df["MachineIdentifier"].nunique()

Podemos ver que en esta columna ocurre algo parecido, todos los valores son distintos y por el nombre de la columna podemos saber que contiene el identificador de los dispositivos, por lo que prescindiremos de esta columna

In [11]:
del_columns.append("MachineIdentifier")

### "ProductName"

In [ ]:
df["ProductName"].value_counts()

In [ ]:
df["ProductName"].value_counts()/df.shape[0]  # càlculo de proporciones

Podemos ver que tenemos 3 valores distintos, pero el valor win8defender aparece 98% de las veces

In [ ]:
df["ProductName"].hist()

#### DELETE para ver

### "EngineVersion"

In [ ]:
df["EngineVersion"].value_counts().sort_index()

Vemos que hay varios valores en esta columna, ahora comprobaremos si el EngineVersion tiene que ver con la columna ProductName

In [ ]:
df[df["ProductName"] == "mse"]["EngineVersion"].value_counts()

In [ ]:
df[df["EngineVersion"] == "1.1.15100.1"]

Por lo que podemos ver no tienen nada que ver por lo que puede llegar a ser interesante mantenerla, además, podríamos elimnar el ultimo número de la versión para poder agruparlas un poco más

In [ ]:
df["EngineVersion"].str.rsplit(".", n=1).str[0]

In [19]:
df["EngineVersion_trimmed"] = df["EngineVersion"].str.rsplit(".", n=1).str[0]
df["EngineVersion_trimmed"] = df["EngineVersion_trimmed"].str.rsplit(".", n=1).str[-1]

del_columns.append("EngineVersion")

In [ ]:
df["EngineVersion_trimmed"].value_counts().sort_index()

### "AppVersion"

In [ ]:
df["AppVersion"].value_counts().sort_index()

In [ ]:
df[df["ProductName"] == "mse"]["AppVersion"].value_counts()

In [ ]:
df[df["AppVersion"] == "4.10.209.0"]

Podemos ver que el AppVersion y el ProductName no tienen relación por lo que haremos lo mismo que con la columna EngineVersion

In [24]:
df["AppVersion_trimmed"] = df["AppVersion"].str.split(".").str[1]

del_columns.append("AppVersion")

In [ ]:
df["EngineVersion_trimmed"].value_counts().sort_index()

### "AvSigVersion"

In [ ]:
df["AvSigVersion"].value_counts().sort_index()

In [ ]:
df.groupby('AvSigVersion')['HasDetections'].mean().sort_values(ascending=False)

Vamos a comprobar cuantos de los valores terminan con ".0"

In [ ]:
df[df["AvSigVersion"].str.endswith(".0")]["AvSigVersion"].count() / df.shape[0]

Con este codigo filtro por los valores de la columna "AvSigVersion" que terminen en ".0" y cuento cuantos valores recupero y lo divido entre el total de filas del dataset.

Y podemos ver que todos los valores de esta columna terminan con ".0" (el 100%), por lo que eliminaré el ".0"

In [29]:
df["AvSigVersion_trimmed"] = df["AvSigVersion"].str.rsplit(".", n=1).str[0]

del_columns.append("AvSigVersion")

In [ ]:
df["AvSigVersion_trimmed"].value_counts().sort_index()

Comprobamos que todos empiezan por 1.

In [ ]:
df[~df['AvSigVersion_trimmed'].str.startswith("1.")]['AvSigVersion_trimmed']

Vemos que no empiezan con 1. y tienen de valor 0.0.0, por lo cual los eliminamos

In [ ]:
df[df['AvSigVersion_trimmed'].str.startswith("1.")]['AvSigVersion_trimmed']

In [ ]:
df = df[df['AvSigVersion_trimmed'].str.startswith("1.")]
df['AvSigVersion_trimmed'] = df['AvSigVersion_trimmed'].apply(lambda x: x.split(".")[1])

### "IsBeta"

In [ ]:
df["IsBeta"].value_counts()

Podemos ver que el valor True de "IsBeta" es tan solo 1 contra el resto de datos, por lo que prescendiremos de esta columna

In [35]:
del_columns.append("IsBeta")

### "RtpStateBitfield"

In [ ]:
df["RtpStateBitfield"].value_counts()

In [ ]:
print(df[["RtpStateBitfield", "HasDetections"]].corr())

Por su baja correlación con el target prescindiremos de esta columna

In [38]:
del_columns.append("RtpStateBitfield")

### "IsSxsPassiveMode"

In [ ]:
df["IsSxsPassiveMode"].value_counts()

In [ ]:
df["IsSxsPassiveMode"].value_counts()/ df.shape[0]

Podemos ver que el valor True de esta columna representa menos del 2% del dataframe

In [ ]:
print(df[["IsSxsPassiveMode", "HasDetections"]].corr())

No esta balanceado y tiene una correlación debil con el target, por lo que prescindiremos de esta columna

In [42]:
del_columns.append("IsSxsPassiveMode")

### "DefaultBrowsersIdentifier"

In [ ]:
df["DefaultBrowsersIdentifier"].value_counts()

In [ ]:
print(df[["DefaultBrowsersIdentifier", "HasDetections"]].corr())

Por la baja correlación con el target, esta columna será eliminada

In [45]:
del_columns.append("DefaultBrowsersIdentifier")

### "AVProductStatesIdentifier"

In [ ]:
df["AVProductStatesIdentifier"].value_counts()

In [ ]:
print(df[["AVProductStatesIdentifier", "HasDetections"]].corr())

Mantendremos la columna temporalmente y probaremos entrenar al modelo con y sin esta columna

Al mirar la info del dataset vimos que esta columna contenia nulos

In [ ]:
df["AVProductStatesIdentifier"].isna().sum()/df.shape[0]

Podemos ver que menos del 1% de los datos en esta columna es nulo, por lo que borraremos las filas que contienen nulo

In [ ]:
df.dropna(subset=["AVProductStatesIdentifier"], inplace=True)

In [ ]:
df["AVProductStatesIdentifier"].hist()

### "AVProductsInstalled"

In [ ]:
df["AVProductsInstalled"].value_counts()

In [ ]:
print(df[["AVProductsInstalled", "HasDetections"]].corr())

Esta variable podría resultar interesante, aun siendo muy poca correlación negativa, es interesante ver que en algunos casos al tener más antivirus instalados pues se detectan menos virus

Anteriormente al mirar la info del dataset, vimos que esta columna tenía nulos

In [ ]:
df["AVProductsInstalled"].isna().sum()

Ahora contiene 0% de nulos, así que podemos suponer que las filas con nulos en la columna "AVProductStatesIdentifier" coincidian con las de esta columna, ya que ambas tenían 1938 nulos

In [ ]:
df["AVProductsInstalled"].hist()

### "AVProductsEnabled"

In [ ]:
df["AVProductsEnabled"].value_counts()

In [ ]:
print(df[["AVProductsEnabled", "HasDetections"]].corr())

In [ ]:
print(df[["AVProductsEnabled", "AVProductsInstalled"]].corr())

Tiene muy baja relación con el target pero es algo más alta (casi 30%) de correlación con AVProductsInstalled, por lo que podemos mantenerla temporalmente y hacer pruebas

In [ ]:
df["AVProductsEnabled"].isna().sum()/df.shape[0]

Ahora contiene 0% de nulos, así que podemos suponer que las filas con nulos en las 2 ultimas columnas coincidian con las de esta columna, ya que las 3 tenían 1938 nulos

In [ ]:
df["AVProductsEnabled"].hist()

### "HasTpm"

In [ ]:
df["HasTpm"].value_counts()

In [ ]:
df["HasTpm"].value_counts()/df.shape[0]

In [ ]:
print(df[["HasTpm", "HasDetections"]].corr())

Al ver que hay un fuerte desequilibrio en cuanto a los valores y que su correlación con el target es muy debil, prescindiremos de la columna

In [63]:
del_columns.append("HasTpm")

### "CountryIdentifier"

In [ ]:
df["CountryIdentifier"].value_counts()

In [ ]:
print(df[["CountryIdentifier", "HasDetections"]].corr())

Por la baja correlación con el target, esta columna será eliminada

In [66]:
del_columns.append("CountryIdentifier")

### "CityIdentifier"

In [ ]:
df["CityIdentifier"].value_counts()

In [ ]:
df["CityIdentifier"].isna().sum()/df.shape[0]

In [ ]:
print(df[["CityIdentifier", "HasDetections"]].corr())

Por la baja correlación con el target, esta columna será eliminada

In [70]:
del_columns.append("CityIdentifier")

### "OrganizationIdentifier"

In [ ]:
df["OrganizationIdentifier"].value_counts()

In [ ]:
df["OrganizationIdentifier"].isna().sum()/df.shape[0]

Podemos ver que el 30% de esta columna es nulo, por lo que antes de valorar si eliminarla o no, comprobaremos su correlación con el target

In [ ]:
print(df[["OrganizationIdentifier", "HasDetections"]].corr())

Al tener una correlación tan baja suponemos que la columna probablemente no tenga un impacto significativo, por lo que prescindiremos de la columna

In [74]:
del_columns.append("OrganizationIdentifier")

### "GeoNameIdentifier"

In [ ]:
df["GeoNameIdentifier"].value_counts()

In [ ]:
print(df[["GeoNameIdentifier", "HasDetections"]].corr())

Por la baja correlación con el target, esta columna será eliminada

In [77]:
del_columns.append("GeoNameIdentifier")

### "LocaleEnglishNameIdentifier"

In [ ]:
df["LocaleEnglishNameIdentifier"].value_counts()

In [ ]:
print(df[["LocaleEnglishNameIdentifier", "HasDetections"]].corr())

Por la baja correlación con el target, esta columna será eliminada

In [80]:
del_columns.append("LocaleEnglishNameIdentifier")

### "Platform"

In [ ]:
df["Platform"].value_counts()

In [ ]:
df["Platform"].hist()

Aunque los valores estan muy desbalanceados, haremos pruebas y según los resultados de estas veremos si mantener o no la columna

El siguiente punto será hacer un OHE con esta columna

In [83]:
df = pd.get_dummies(df, columns=["Platform"], prefix="Platform")

### "Processor"

In [ ]:
df["Processor"].value_counts()

In [ ]:
df["Processor"].hist()

Aunque los valores estan desbalanceados, haremos pruebas y según los resultados de estas veremos si mantener o no la columna

El siguiente punto será hacer un OHE con esta columna

In [86]:
df = df[df["Processor"]!="arm64"]

In [87]:
df['Processor'] = df['Processor'].apply(lambda x: 1 if x=="x86" else 0)

## Parte MARC


In [ ]:
list(df.columns[42:63])

Columnas a revisar:
['Census_ProcessorClass',
 'Census_PrimaryDiskTotalCapacity',
 'Census_PrimaryDiskTypeName',
 'Census_SystemVolumeTotalCapacity',
 'Census_HasOpticalDiskDrive',
 'Census_TotalPhysicalRAM',
 'Census_ChassisTypeName',
 'Census_InternalPrimaryDiagonalDisplaySizeInInches',
 'Census_InternalPrimaryDisplayResolutionHorizontal',
 'Census_InternalPrimaryDisplayResolutionVertical',
 'Census_PowerPlatformRoleName',
 'Census_InternalBatteryType',
 'Census_InternalBatteryNumberOfCharges',
 'Census_OSVersion',
 'Census_OSArchitecture',
 'Census_OSBranch',
 'Census_OSBuildNumber',
 'Census_OSBuildRevision',
 'Census_OSEdition',
 'Census_OSSkuName',
 'Census_OSInstallTypeName']

In [ ]:
len(df)

### "Census_ProcessorClass"
Descripción: "Clase" del procesador del dispositivo

In [ ]:
df["Census_ProcessorClass"].value_counts(dropna=False)

In [91]:
# 497_918 / 500_000 son nulos, por lo que eliminaremos la columna
del_columns.append("Census_ProcessorClass")

### "Census_PrimaryDiskTotalCapacity"
Descripción: Capacidad total del disco duro primario en MB

In [ ]:
df["Census_PrimaryDiskTotalCapacity"].value_counts(dropna=False)

In [ ]:
# contamos nulos
df["Census_PrimaryDiskTotalCapacity"].isna().sum()  # 2968 nulos

In [ ]:
# miramos la correlación con el target
df[["Census_PrimaryDiskTotalCapacity", "HasDetections"]].corr()

Difícilmente la capacidad del disco duro puede tener relación con la detección de malware, por lo que eliminaremos la columna.

In [95]:
del_columns.append("Census_PrimaryDiskTotalCapacity")

### "Census_PrimaryDiskTypeName"
Descripción: Tipo de disco duro primario

In [ ]:
df["Census_PrimaryDiskTypeName"].value_counts(dropna=False)

In [97]:
# juntamos valores poco frecuentes (UNKKNOWN, Unspecified) a "Indefinido"
df["Census_PrimaryDiskTypeName"] = df["Census_PrimaryDiskTypeName"].replace(["UNKNOWN", "Unspecified"], "Indefinido")
# rellenamos nulos con "Indefinido"
df["Census_PrimaryDiskTypeName"] = df["Census_PrimaryDiskTypeName"].fillna("Indefinido")

In [ ]:
df["Census_PrimaryDiskTypeName"].value_counts(dropna=False)

Se propone realizar un OHE con esta columna.

In [99]:
df = pd.get_dummies(df, columns=["Census_PrimaryDiskTypeName"], prefix="Census_PrimaryDiskTypeName")  # TODO: revisar

### "Census_SystemVolumeTotalCapacity"
Descripción: Capacidad total del volumen del sistema en MB

In [ ]:
df["Census_SystemVolumeTotalCapacity"].value_counts(dropna=False)

In [ ]:
# hacemos un histograma
df[df["Census_SystemVolumeTotalCapacity"] < 1_000_000]["Census_SystemVolumeTotalCapacity"].hist()

In [ ]:
# correlación con el target
df[["Census_SystemVolumeTotalCapacity", "HasDetections"]].corr()

In [103]:
# mismo argumento que "Census_PrimaryDiskTotalCapacity", eliminamos la columna
del_columns.append("Census_SystemVolumeTotalCapacity")

### "Census_HasOpticalDiskDrive"
Descripción: Indica si el dispositivo tiene una unidad de disco óptico

In [ ]:
df["Census_HasOpticalDiskDrive"].value_counts(dropna=False)

In [ ]:
38_517 / 500_000  # 7.7% True

In [ ]:
# correlación con el target
df[["Census_HasOpticalDiskDrive", "HasDetections"]].corr()

In [107]:
# TODO: Muy poca correlación, la dejamos pero la podríamos borrar perfectamente (dependerá de los resultados de los modelos)

### "Census_TotalPhysicalRAM"
Descripción: Cantidad de RAM física en MB

In [ ]:
df["Census_TotalPhysicalRAM"].value_counts(dropna=False)

In [ ]:
#  pasamos a GB
df["Census_TotalPhysicalRAM"].apply(lambda x: x // 1024).value_counts().sort_index()

In [110]:
"""
Opciones:
1. Eliminar la columna por (alomejor) no tener relación con el target
2. Eliminamos los registros con valores "anormales" o "poco frecuentes"
3. Otra (a definir)
"""

# OPCIÓN 2!
# convertimos a GB
df["Census_TotalPhysicalRAM_GB"] = df["Census_TotalPhysicalRAM"].apply(lambda x: x // 1024)

In [ ]:
# nulos
df["Census_TotalPhysicalRAM_GB"].isna().sum()  # 4556 nulos

In [112]:
# Buscamos aquellos valores de RAM que tengan más de 1000 filas en caso de RAM < 20 GB y de más de 100 filas en caso de RAM >= 20 GB
recuento_RAM = df["Census_TotalPhysicalRAM_GB"].value_counts().sort_index()
a = recuento_RAM.iloc[:20][recuento_RAM.iloc[:20] > 1000] # RAM < 20 GB
b = recuento_RAM.iloc[20:][recuento_RAM.iloc[20:] > 100] # RAM >= 20 GB

In [ ]:
ram_elecciones = a.index.tolist() + b.index.tolist()
ram_elecciones  # valores más frecuentes, nos quedamos con los registros que contenegan estos valores

In [114]:
# filtramos por los registros que tengan la RAM que hemos seleccionado
df = df[df["Census_TotalPhysicalRAM_GB"].isin(ram_elecciones)]  # indirecamente eliminamos los nulos

In [ ]:
print(f"Recuento de registros actuales: {len(df)}")  # 493_254, 6746 menos
df["Census_TotalPhysicalRAM_GB"].value_counts()

In [ ]:
# correlación con el target
df[["Census_TotalPhysicalRAM_GB", "HasDetections"]].corr()

In [117]:
del_columns.append("Census_TotalPhysicalRAM")

### "Census_ChassisTypeName"
Descripción: Tipo de chasis del dispositivo

In [ ]:
df["Census_ChassisTypeName"].value_counts(dropna=False)

In [ ]:
# Convertir la variable string en variables dummy
df_dummies = pd.get_dummies(df['Census_ChassisTypeName'], prefix='Census_ChassisTypeName')

# Calcular la correlación de cada variable dummy con el target
correlations = df_dummies.corrwith(df['HasDetections'])

correlations

In [120]:
# Correlaciones < 0.04 -> proponemos eliminar la columna
del_columns.append("Census_ChassisTypeName")

In [ ]:
df.head(2)

### "Census_InternalPrimaryDiagonalDisplaySizeInInches"
Descripción: Tamaño diagonal de la pantalla en pulgadas

In [ ]:
df["Census_InternalPrimaryDiagonalDisplaySizeInInches"].value_counts(dropna=False)

In [ ]:
# hist
df["Census_InternalPrimaryDiagonalDisplaySizeInInches"].hist()

In [124]:
# se deduce una muy baja correlación con el target, eliminamos la columna  # TODO
del_columns.append("Census_InternalPrimaryDiagonalDisplaySizeInInches")

In [ ]:
# correlación con el target
df[["Census_InternalPrimaryDiagonalDisplaySizeInInches", "HasDetections"]].corr()

### "Census_InternalPrimaryDisplayResolutionHorizontal" y "Census_InternalPrimaryDisplayResolutionVertical"
Descripción: Resolución de la pantalla

In [ ]:
df["Census_InternalPrimaryDisplayResolutionHorizontal"].value_counts(dropna=False)

In [ ]:
df["Census_InternalPrimaryDisplayResolutionVertical"].value_counts(dropna=False)

In [128]:
# misma argumentación que "Census_InternalPrimaryDiagonalDisplaySizeInInches", proponemos eliminar las columnas
del_columns.append("Census_InternalPrimaryDisplayResolutionHorizontal")
del_columns.append("Census_InternalPrimaryDisplayResolutionVertical")

### "Census_PowerPlatformRoleName"
Descripción: Nombre del rol de la plataforma de energía

In [ ]:
df["Census_PowerPlatformRoleName"].value_counts(dropna=False)  # no tiene nulos

In [130]:
# juntamos columnas [SOHOServer, EnterpriseServer, PerformanceServer] en "Server"
df["Census_PowerPlatformRoleName"] = df["Census_PowerPlatformRoleName"].replace(["SOHOServer", "EnterpriseServer", "PerformanceServer"], "Server")

In [131]:
# borramos registros con nulos (con valor UNKNOWN)
df = df[df["Census_PowerPlatformRoleName"] != "UNKNOWN"]

In [ ]:
df["Census_PowerPlatformRoleName"].value_counts(dropna=False)

In [133]:
# hay únicamente 9 valores únicos, es una variable categórica, proponemos hacer un OHE
df = pd.get_dummies(df, columns=["Census_PowerPlatformRoleName"], prefix="Census_PowerPlatformRoleName")  # TODO

### "Census_InternalBatteryType"
Descripción: Tipo de batería interna

In [ ]:
df["Census_InternalBatteryType"].value_counts(dropna=False)

In [135]:
# interpeteción: hay 348_471 nulos y se puede presuponer una alta correlación entre el tipo de batería y el tipo de dispositivo -> eliminamos la columna #TODO: comprobar
del_columns.append("Census_InternalBatteryType")

### "Census_InternalBatteryNumberOfCharges"
Descripción: Número de cargas de la batería interna

In [ ]:
df["Census_InternalBatteryNumberOfCharges"].value_counts(dropna=False)

In [137]:
# 1. se presupone que serà una variable contínua
# 2. nulos: hay directamente 12204 NaNs (2.5%) pero se observa unos registros extremadamente altos de 4294967295 y 4294967294. Estos valores provienen de un error en la recogida de datos, 4294967295 es el valor máximo de un entero de 32 bits (2^32 - 1). Hay 123_658 y 656 registros respectivamente (24.9% y 0.1%). Demasiados registros para eliminarlos, se propone rellenar los nulos con la media de la columna.

# TODO: exponer al grupo
# calculamos media evitando los valores erróneos

df["Census_InternalBatteryNumberOfCharges"] = df["Census_InternalBatteryNumberOfCharges"].fillna(0)

In [ ]:
# vamos los registros fallidos
df[df["Census_InternalBatteryNumberOfCharges"] > 4_000_000_000]
# observamos que los registros con valor 4294967295 son los que tienen nulos en la columna "Census_InternalBatteryType" -> comportamiento esperado

In [139]:
# se imputan los valores erróneos con la media
mask = df["Census_InternalBatteryNumberOfCharges"] > 4_000_000_000
df.loc[mask, "Census_InternalBatteryNumberOfCharges"] = 0

In [ ]:
# hist
df["Census_InternalBatteryNumberOfCharges"].hist()

In [ ]:
df['Census_InternalBatteryNumberOfCharges'].value_counts(dropna=False)

In [ ]:
# miramos registros con valor 0
df[df["Census_InternalBatteryNumberOfCharges"] == 0]

In [ ]:
# correlación con el target
df[["Census_InternalBatteryNumberOfCharges", "HasDetections"]].corr()

In [144]:
# es una columna que indirectamente nos puede estar devolviendo en tiempo de uso del dispositivo, proponemos mantenerla aún habiendo 280_000 registros con valor 0 (56%). Es la única que nos proporciona esta información. # TODO: debatir mas a fondo

In [145]:
del_columns.append("Census_InternalBatteryNumberOfCharges")

### "Census_OSVersion"
Descripción: Versión del sistema operativo

In [ ]:
print(f"{df['Census_OSVersion'].isna().sum()=}")  # 0 nulos
df["Census_OSVersion"].value_counts(dropna=False)
# Únicamente hay un registro con la versión distinta a la 10.0... siendo la 6.3.9600.19069

In [ ]:
df[df['OsVer'] == '6.3.0.0'][['OsVer', 'Census_OSVersion']]

In [148]:
# se podía esperar una fueran la misma columna, pero este ejemplo nos demuestra que no es así
# siendo todos los valores 10.0... podemos prononer a lo sumo quedarnos con "el tercer valor" de la versión que podría estar relacionado con parches de seguridad (p.e.) # TODO: Exponer el caso
# #TODO: Propongo que los valores "muy manipulados" empiecen por CALC_

# evitamos el caso del registro 6.3.9600.19069 eliminándolo
df = df[df['Census_OSVersion'] != '6.3.9600.19069']
df["CALC_Census_OSVersion"] = df["Census_OSVersion"].apply(lambda x: x.split(".")[2])

In [149]:
# eliminamos la columna original
del_columns.append("Census_OSVersion")

In [ ]:
df["CALC_Census_OSVersion"].value_counts(dropna=False)

### "Census_OSArchitecture"
Descripción: Arquitectura del sistema operativo

In [ ]:
df["Census_OSArchitecture"].value_counts(dropna=False)  # dos valores únicos (amd64 y x86) y sin nulos

In [ ]:
# comprobamos correlación con la columna processor_x64 y processor_x86
df[df["Processor"] == True]["Census_OSArchitecture"].value_counts()  # coincide en un 446_028/(446_028+452) = 99.9%

In [ ]:
df[df["Processor"] == True]["Census_OSArchitecture"].value_counts()  # coincide en un 44555/(44555+459) = 99.9%

In [154]:
# TODO: puesto que esta columna no es nada conlfictiva, tengo que preguntar que tal se comportó la otra columna (processor) preguntar a fran
# eliminamos la columna por ser redundante
del_columns.append("Census_OSArchitecture")

### "Census_OSBranch"
Descripción: Rama del sistema operativo

In [ ]:
df["Census_OSBranch"].value_counts(dropna=False)

In [ ]:
print(f"{df['Census_OSBranch'].isna().sum()=}")  # 0 nulos

In [157]:
# viendo los valores únicos y su estructura, proponemos quedarnos con las 3 primeras letras de cada valor que parecen contener la información más relevante
df["CALC_Census_OSBranch"] = df["Census_OSBranch"].apply(lambda x: x[:3])

In [ ]:
df["CALC_Census_OSBranch"].value_counts(dropna=False)

In [159]:
# modificamos el valor menos frecuente ("rs_") a "rs"
df["CALC_Census_OSBranch"] = df["CALC_Census_OSBranch"].replace("rs_", "rs")  # TODO: hacer el split por _ DEBERES

In [160]:
# eliminamos la columna original
del_columns.append("Census_OSBranch")

In [ ]:
df["CALC_Census_OSBranch"].value_counts(dropna=False)

In [162]:
# 8 valores únicos de una variable categórica -> OHE
df = pd.get_dummies(df, columns=["CALC_Census_OSBranch"], prefix="CALC_Census_OSBranch")  # TODO

### "Census_OSBuildNumber"
Descripción: Número de compilación del sistema operativo

In [ ]:
print(f"{df['Census_OSBuildNumber'].isna().sum()=}")  # 0 nulos
df["Census_OSBuildNumber"].value_counts(dropna=False)

In [ ]:
df[['CALC_Census_OSVersion', 'Census_OSBuildNumber']].dtypes

In [ ]:
df[['CALC_Census_OSVersion', 'Census_OSBuildNumber']]

In [166]:
# viendo que la columna préviamente creada "CALC_Census_OSVersion" contiene la misma información que "Census_OSBuildNumber", eliminamos la columna
del_columns.append("CALC_Census_OSVersion") # TODO: pedir al grupo si mantener el código aún sabiendo que es redundante

### "Census_OSBuildRevision"
Descripción: Revisión de la compilación del sistema operativo

In [ ]:
print(f"{df['Census_OSBuildRevision'].isna().sum()=}")  # 0 nulos
df["Census_OSBuildRevision"].value_counts(dropna=False)

In [ ]:
df.head(2)

In [169]:
# se observa que Census_OSVersion se construye a partir de Census_OSBuildNumber y Census_OSBuildRevision, eliminamos la columna
# proponemos eliminar la columna Census_OSVersion (ya insertada en del_columns) porque es un str y las otras dos columnas son numéricas

In [170]:
del_columns.append("Census_OSBuildRevision")

### "Census_OSEdition"
Descripción: Edición del sistema operativo

In [ ]:
print(f"{df['Census_OSEdition'].isna().sum()=}")  # 0 nulos
df["Census_OSEdition"].value_counts(dropna=False)

In [172]:
# hay 3 valores muy predominantes con 192000, 171000 y 107000 registros respectivamente (38.9%, 34.7% y 21.7%). Se propone agrupar los 16 valores restantes en un solo valor "Otro" y posteriormente hacer un OHE
# agrupamos los valores menos frecuentes
df["Census_OSEdition"] = df["Census_OSEdition"].replace(df["Census_OSEdition"].value_counts().index[3:], "Otro")

In [ ]:
df["Census_OSEdition"].value_counts(dropna=False)

In [174]:
# la columna otros contiene 19827 registros (4%)
# OHE
df = pd.get_dummies(df, columns=["Census_OSEdition"], prefix="Census_OSEdition")  # TODO

### "Census_OSSkuName"
Descripción: Nombre de la versión del sistema operativo

In [ ]:
print(f"{df['Census_OSSkuName'].isna().sum()=}")  # 0 nulos
df["Census_OSSkuName"].value_counts(dropna=False)

In [176]:
# es un atributo prácticamente idéntico a Census_OSEdition, eliminamos la columna
del_columns.append("Census_OSSkuName")

### "Census_OSInstallTypeName"
Descripción: Método de instalación del sistema operativo

In [ ]:
print(f"{df['Census_OSInstallTypeName'].isna().sum()=}")  # 0 nulos
df["Census_OSInstallTypeName"].value_counts(dropna=False)

In [ ]:
# puede parecer difícilmente relacionable con el target, comprobamos correlaciones
df_dummies = pd.get_dummies(df['Census_OSInstallTypeName'], prefix='Census_OSInstallTypeName')
correlations = df_dummies.corrwith(df['HasDetections'])
correlations

In [179]:
# correlaciones no superiores al 0.04 -> eliminamos la columna
del_columns.append("Census_OSInstallTypeName")

In [ ]:
df['Census_FirmwareManufacturerIdentifier'].value_counts().sort_values(ascending=False).index[:6].to_list()

In [ ]:
df['Census_FirmwareVersionIdentifier'].value_counts().sort_values(ascending=False)

In [ ]:
df.info()

## Parte IGNACIO

In [ ]:
df.info()

In [ ]:
valores_faltantes = (df.isnull().mean() * 100).sort_values(ascending=False)
print(valores_faltantes)

In [ ]:
((df.isna().sum()/df.shape[0])*100).sort_values().plot(kind="barh",figsize=(10,10),color="lightblue")
plt.title('Porcentaje de valores faltantes')
plt.xlabel('Porcentaje del total de filas')
plt.ylabel('Variable')
plt.xlim(0, 100)
plt.show() 

In [186]:
df = df.drop(columns=['PuaMode', 'SMode', 'SmartScreen', 'OsBuild', 'OsPlatformSubRelease', 'OsBuildLab', 'AutoSampleOptIn', 'Census_MDC2FormFactor', 'Census_DeviceFamily', 'Census_OEMModelIdentifier'])

In [187]:
columnas_objetivo_ict = ['Census_OEMNameIdentifier','Firewall','IeVerIdentifier','Census_ProcessorModelIdentifier','Census_ProcessorCoreCount','Census_ProcessorManufacturerIdentifier','IsProtected','UacLuaenable']
df = df.dropna(subset=columnas_objetivo_ict)

df = df[~df["UacLuaenable"].isin(list([48.0, 6357062.0000, 2.0]))]
df = df[df['Census_ProcessorManufacturerIdentifier'] != 3.0]

In [ ]:
df["OsVer"].value_counts()

In [189]:
df['OsVer'] = df['OsVer'].apply(lambda x: x.split('.')[0])

In [ ]:
df['OsSuite'].value_counts()

In [191]:
df = df[df['OsSuite'] != 784]

In [ ]:
df.info()

## Parte ALVARO

In [ ]:
df[alvaro_columns].isna().sum()

---
### Census_IsFlightingInternal, Census_ThresholdOptIn y Census_IsWIMBootEnabled.

##### Primero quiero visualizar el porcentaje de valores nulos por columna

In [ ]:

(df[alvaro_columns].isnull().sum() / len(df[alvaro_columns])) * 100


#####  Lo primero que estoy viendo es que hay columnas con un 50% de nulos, estas son `Census_IsFlightingInternal`, `Census_ThresholdOptIn` y `Census_IsWIMBootEnabled`.
#####  Por lo tanto voy a proceder a prescindir de estas.

In [195]:
del_columns.extend(["Census_IsFlightingInternal", "Census_ThresholdOptIn", "Census_IsWIMBootEnabled"])

---
### Census_OSInstallLanguageIdentifier

In [ ]:
df["Census_OSInstallLanguageIdentifier"].value_counts()

Vamos a ver que grado de correlacion tiene con el target y asi saber si podemos descartarla con seguridad.

In [ ]:
df[["Census_OSInstallLanguageIdentifier", "HasDetections"]].corr()

##### Podemos ver que la correlacion es de menos de un 0.001%, por lo tanto es seguro descartarla.

In [198]:
del_columns.append('Census_OSInstallLanguageIdentifier')

---
### Census_OSUILocaleIdentifier

In [ ]:
df["Census_OSUILocaleIdentifier"].value_counts()

En base de la correlacion que mantenga con el target, decidiremos si prescindimos de esta columna o no.

In [ ]:
df[["Census_OSUILocaleIdentifier", "HasDetections"]].corr()

##### Al igual que con la anterior columna, no llega ni al 1% de correlacion, por lo tanto vamos a eliminar esta columna.

In [201]:
del_columns.append('Census_OSUILocaleIdentifier')

---
### Census_OSWUAutoUpdateOptionsName

In [ ]:
df["Census_OSWUAutoUpdateOptionsName"].value_counts()

Esta columna tiene mas sentido mantenerla por que no tiene muchos valores y tiene mas sentido que algo relacionado con actualizaciones del OS tenga relacion con el target. A pesar de que tenemos aproximadamente un 25% de valores con `UNKNOWN` veremos que pasa en los tests y en funcion de eso volveremos para eliminarla o no.

---
### Census_IsPortableOperatingSystem

In [ ]:
df["Census_IsPortableOperatingSystem"].value_counts()

##### Esta columna ademas de no tener nulos (como hemos visto anteriormente) solo contiene 0 y 1.

In [ ]:
df[["Census_IsPortableOperatingSystem", "HasDetections"]].corr()

Lo unico que no me convence es que la correlacion es muy baja. Eliminaremos esta columna ya que no afectara mucho al entrenamiendo del modelo.

In [205]:
del_columns.append('Census_IsPortableOperatingSystem')

---
### Census_GenuineStateName

Borraremos las obeservaciones que tenga valor `UNKNOWN` ya que asi nos ahorraremos una columna categorica.

In [ ]:
df[~df["Census_GenuineStateName"].isin(["UNKNOWN"])]["Census_GenuineStateName"].value_counts()

Aunque los valores estan desbalanceados, haremos pruebas y según los resultados de estas veremos si mantener o no la columna

El siguiente punto será hacer un OHE con esta columna

In [207]:
df = df[df["Census_GenuineStateName"] != "UNKNOWN"]

---
### Census_ActivationChannel

In [ ]:
df["Census_ActivationChannel"].value_counts()

In [ ]:
df["Census_ActivationChannel"].hist()

Podemos ver que hay dos valores que predominan. Podriamos prescindir de las pocas observaciones con valores `Volume:MAK` y `Retail:TB:Eval` para asi tener dos columnas catergoricas menos al hacer OHE

In [ ]:
df[~df["Census_ActivationChannel"].isin(["Volume:MAK", "Retail:TB:Eval"])]["Census_ActivationChannel"].value_counts()

---
### Census_IsFlightsDisabled

In [ ]:
df["Census_IsFlightsDisabled"].value_counts()

In [ ]:
df[["Census_IsFlightsDisabled", "HasDetections"]].corr()

A parte de que la varianza es casi inexistente, la correlacion es tambien muy baja, por lo tanto eliminaremos esta columna.

In [213]:
del_columns.append('Census_IsFlightsDisabled')

---
### Census_FlightRing

In [ ]:
df["Census_FlightRing"].value_counts()

In [ ]:
df["Census_FlightRing"].hist()

Aqui podemos apreciar valores muy desbalanceados y como poca variabilidad. Vamos a eliminar esta columna.

In [216]:
del_columns.append('Census_FlightRing')

---
### Census_FirmwareManufacturerIdentifier

In [ ]:
df["Census_FirmwareManufacturerIdentifier"].value_counts().sort_index()

In [ ]:
df["Census_FirmwareManufacturerIdentifier"].value_counts().sort_values(ascending=False)

In [ ]:
df["Census_FirmwareManufacturerIdentifier"].hist()

In [220]:
# Obtener las seis categorías más frecuentes
top_6_categories = df["Census_FirmwareManufacturerIdentifier"].value_counts().nlargest(6).index

# Renombrar las categorías y agrupar las restantes bajo "Otros"
df["Census_FirmwareManufacturerIdentifier"] = df["Census_FirmwareManufacturerIdentifier"].apply(
    lambda x: f"Id_{x}" if x in top_6_categories else "Otros"
)

# Realizar One-Hot Encoding
df = pd.get_dummies(df, columns=["Census_FirmwareManufacturerIdentifier"], prefix="FirmwareManufacturer", prefix_sep="")

In [ ]:
df[[
    'FirmwareManufacturerOtros',
    'FirmwareManufacturerId_142.0',
    'FirmwareManufacturerId_355.0',
    'FirmwareManufacturerId_500.0',
    'FirmwareManufacturerId_554.0',
    'FirmwareManufacturerId_556.0',
    'FirmwareManufacturerId_628.0']].corrwith(df['HasDetections'])

In [ ]:
corr, p_value = pointbiserialr(df['FirmwareManufacturerId_142.0'], df['HasDetections'])
print(f'Correlación: {corr}, p-valor: {p_value}')

Al final hemos decidido mantener la columna como un ID y veremos cuanto ayuda al modelo.

---
### Census_FirmwareVersionIdentifier

In [ ]:
df["Census_FirmwareVersionIdentifier"].value_counts().sort_index()

In [ ]:
df["Census_FirmwareVersionIdentifier"].value_counts().sort_values(ascending=False)

In [ ]:
df["Census_FirmwareVersionIdentifier"].hist()

In [ ]:
df["Census_FirmwareVersionIdentifier"].nunique()

In [ ]:
df[["Census_FirmwareVersionIdentifier", "HasDetections"]].corr()

Una vez mas, tiene poca correlacion. Prescindimos de ella.

In [228]:
del_columns.append('Census_FirmwareVersionIdentifier')

---
### Census_IsSecureBootEnabled

In [ ]:
df["Census_IsSecureBootEnabled"].value_counts().sort_index()

In [ ]:
df[["Census_IsSecureBootEnabled", "HasDetections"]].corr()

In [ ]:
df["Census_IsSecureBootEnabled"].hist()

Aqui podemos ver que la relacion con el target es muy baja. Eliminaremos esta columna.

In [232]:
del_columns.append('Census_IsSecureBootEnabled')

---
### Census_IsVirtualDevice

In [ ]:
df["Census_IsVirtualDevice"].value_counts().sort_index()

In [ ]:
df["Census_IsVirtualDevice"].hist()

In [ ]:
df[["Census_IsVirtualDevice", "HasDetections"]].corr()

Siendo la correlacion tan baja y el balance de datos tan descompensado tambien vamos a eliminar esta columna.

In [236]:
del_columns.append('Census_IsVirtualDevice')

---
### Census_IsTouchEnabled

In [ ]:
df["Census_IsTouchEnabled"].value_counts().sort_index()

In [ ]:
df["Census_IsTouchEnabled"].hist()

In [ ]:
df[["Census_IsTouchEnabled", "HasDetections"]].corr()

No hay mucha correlacion con el target y los valores estan descompensados. Eliminamos la columna.

In [240]:
del_columns.append('Census_IsTouchEnabled')

---
### Census_IsPenCapable

In [ ]:
df["Census_IsPenCapable"].value_counts().sort_index()

In [ ]:
df["Census_IsPenCapable"].hist()

In [ ]:
df[["Census_IsPenCapable", "HasDetections"]].corr()

No hay mucha correlacion con el target y los valores estan descompensados. Eliminamos la columna.

In [244]:
del_columns.append('Census_IsPenCapable')

---
### Census_IsAlwaysOnAlwaysConnectedCapable

In [ ]:
df["Census_IsAlwaysOnAlwaysConnectedCapable"].value_counts().sort_index()

In [ ]:
df["Census_IsAlwaysOnAlwaysConnectedCapable"].hist()

In [ ]:
df[["Census_IsAlwaysOnAlwaysConnectedCapable", "HasDetections"]].corr()

No hay mucha correlacion con el target y los valores estan descompensados. Eliminamos la columna.

In [248]:
del_columns.append('Census_IsAlwaysOnAlwaysConnectedCapable')

---
### Wdft_IsGamer

In [ ]:
df["Wdft_IsGamer"].value_counts().sort_index()

In [ ]:
df["Wdft_IsGamer"].hist()

In [ ]:
df[["Wdft_IsGamer", "HasDetections"]].corr()

No hay mucha correlacion con el target y los valores estan descompensados. Eliminamos la columna.

In [252]:
del_columns.append('Wdft_IsGamer')

---
### Wdft_RegionIdentifier

In [ ]:
df["Wdft_RegionIdentifier"].value_counts().sort_index()

In [254]:
# Calcular el umbral del 5%
threshold = len(df) * 0.05

# Obtener los valores que superan el umbral
top_region_ids = df["Wdft_RegionIdentifier"].value_counts()[df["Wdft_RegionIdentifier"].value_counts() > threshold].index

# Renombrar las categorías y agrupar las restantes bajo "OtrosRegionId"
df["Wdft_RegionIdentifier"] = df["Wdft_RegionIdentifier"].apply(
    lambda x: f"RegionId_{x}" if x in top_region_ids else "OtrosRegionId"
)

# Realizar One-Hot Encoding
df = pd.get_dummies(df, columns=["Wdft_RegionIdentifier"], prefix="", prefix_sep="")

In [ ]:
df.head()

Aqui se podria hcer el mismo procedimiento que hemos hecho con `Census_FirmwareManufacturerIdentifier` o se borra directamente. Para hablarlo.

In [ ]:
df[['OtrosRegionId','RegionId_1.0','RegionId_10.0','RegionId_11.0','RegionId_15.0','RegionId_3.0','RegionId_7.0']].corrwith(df['HasDetections'])

In [ ]:
df.info()

## Borramos las columnas que no necesitamos

In [258]:
df.drop(del_columns, axis=1, inplace=True)

In [259]:
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df

In [ ]:
df.info()

## Modificamos algunas columnas

In [262]:
df['OsSuite'] = df['OsSuite'].apply(lambda x: 1 if x==768 else 0)
df['SkuEdition'] = df['SkuEdition'].apply(lambda x: "Other" if x not in list(["Home", "Pro"]) else x)
df['IeVerIdentifier'] = df['IeVerIdentifier'].apply(lambda x: "Other" if x not in list([137.0000, 117.0000, 108.0000, 111.0000]) else x)
df['Census_OEMNameIdentifier'] = df['Census_OEMNameIdentifier'].apply(lambda x: "Other" if x not in list([2668.0000, 2102.0000, 1443.0000, 2206.0000, 585.0000, 525.0000]) else x)
df['Census_ProcessorCoreCount'] = df['Census_ProcessorCoreCount'].apply(lambda x: "Other" if x not in list([4.0, 2.0, 8.0]) else x)
df['Census_ActivationChannel'] = df['Census_ActivationChannel'].apply(lambda x: x.split(":")[0])

## Census_ProcessorModelIdentifier normalizamos

## Census_OSBuildNumber normalizamos

In [ ]:
df.hist(bins=30, figsize=(15, 40), layout=(19, 3), edgecolor='black', color='skyblue')
plt.suptitle("Distribuciones de las Variables", fontsize=11)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
sns.countplot(data=df, x='HasDetections', palette=['skyblue', 'salmon'], edgecolor='black')

plt.title("Distribución de la Variable Target", fontsize=14)
plt.xlabel("HasDetections (0 = No infectado, 1 = Infectado)", fontsize=12)
plt.ylabel("Frecuencia", fontsize=12)
plt.show()

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Asegurar que las columnas categóricas sean de tipo string
df = df.astype(str)

# Aplicar Label Encoding solo a las columnas categóricas
label_encoder = LabelEncoder()
for column in df.select_dtypes(include=['object']).columns:
    df[column] = label_encoder.fit_transform(df[column])

# Convertir el DataFrame a tipo numérico después del Label Encoding
df = df.apply(pd.to_numeric)

# Normalizar solo las columnas numéricas
scaler = StandardScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

# Crear la matriz de correlación
correlation_matrix = df_normalized.corr()

# Ajustar dinámicamente el tamaño de la figura basado en el número de filas y columnas
n = correlation_matrix.shape[0]
width = max(12, n)
height = max(10, n)

plt.figure(figsize=(width, height))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", square=True, linewidths=0.5)
plt.title(f"Matriz de Correlación ({n}x{n})")
plt.show()



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt

# Dividir los datos en conjuntos de entrenamiento y prueba
X = df.drop('HasDetections', axis=1)  # Eliminar la columna objetivo para obtener solo las características (features)
y = df['HasDetections']  # La variable objetivo que queremos predecir
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)  # 70% entrenamiento, 30% prueba, semilla aleatoria fija para reproducibilidad

# Crear y entrenar el modelo de Decision Tree
tree_model = DecisionTreeClassifier(random_state=42)  # Modelo de árbol de decisión con configuración predeterminada
tree_model.fit(X_train, y_train)  # Entrenar el modelo con los datos de entrenamiento

# Evaluar el modelo en los datos de entrenamiento y prueba
train_accuracy = accuracy_score(y_train, tree_model.predict(X_train))  # Precisión en entrenamiento
test_accuracy = accuracy_score(y_test, tree_model.predict(X_test))  # Precisión en prueba

# Imprimir las métricas de precisión
print(f'Train Accuracy: {train_accuracy}')  # Si la precisión es muy alta aquí, puede indicar sobreajuste
print(f'Test Accuracy: {test_accuracy}')  # Si es mucho menor que la de entrenamiento, el modelo no generaliza bien

# Graficar las curvas de aprendizaje
train_sizes = [1, 100, 500, 2000, 5000, 10000, len(X_train)]  # Diferentes tamaños de entrenamiento para ver el aprendizaje
train_scores = []  # Lista para almacenar la precisión en entrenamiento
test_scores = []  # Lista para almacenar la precisión en prueba

# Entrenar y evaluar el modelo con diferentes tamaños de entrenamiento
for train_size in train_sizes:
    X_train_subset = X_train[:train_size]  # Tomar una porción del conjunto de entrenamiento
    y_train_subset = y_train[:train_size]  # Tomar las etiquetas correspondientes
    tree_model.fit(X_train_subset, y_train_subset)  # Volver a entrenar con el subconjunto reducido
    train_scores.append(accuracy_score(y_train_subset, tree_model.predict(X_train_subset)))  # Precisión en entrenamiento
    test_scores.append(accuracy_score(y_test, tree_model.predict(X_test)))  # Precisión en prueba (se mantiene fija)

# Graficar las curvas de aprendizaje
plt.figure(figsize=(10, 6))  # Ajustar el tamaño de la figura
plt.plot(train_sizes, train_scores, label='Train Accuracy')  # Curva de precisión en entrenamiento
plt.plot(train_sizes, test_scores, label='Test Accuracy')  # Curva de precisión en prueba
plt.xlabel('Training Set Size')  # Etiqueta del eje X
plt.ylabel('Accuracy')  # Etiqueta del eje Y
plt.title('Learning Curves (Decision Tree)')  # Título del gráfico
plt.legend()  # Mostrar leyenda
plt.show()  # Mostrar la gráfica


---
> He hecho algunos ajustes para obtener mejores resultados

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt

# Dividir los datos en conjuntos de entrenamiento y prueba
X = df.drop('HasDetections', axis=1)
y = df['HasDetections']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Obtener valores de poda (ccp_alpha) con muestreo reducido
tree = DecisionTreeClassifier(random_state=42)
path = tree.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas = path.ccp_alphas

# Filtrar valores negativos y reducir la cantidad de valores de ccp_alpha
ccp_alphas = ccp_alphas[ccp_alphas >= 0]
ccp_alphas = np.unique(np.round(ccp_alphas, decimals=6))  # Redondear para evitar valores redundantes
ccp_alphas = np.linspace(min(ccp_alphas), max(ccp_alphas), num=10)  # Seleccionar solo 10 valores para evaluar

# Seleccionar el mejor ccp_alpha con menos cálculos
if len(ccp_alphas) > 0:
    alpha_scores = []

    for alpha in ccp_alphas:
        tree = DecisionTreeClassifier(random_state=42, ccp_alpha=alpha)
        scores = cross_val_score(tree, X_train, y_train, cv=3)  # Reducir a cv=3 para mayor velocidad
        alpha_scores.append(scores.mean())

    best_alpha = ccp_alphas[np.argmax(alpha_scores)]
else:
    best_alpha = 0.0

# Crear y entrenar el modelo optimizado
tree_model = DecisionTreeClassifier(
    max_depth=5,
    min_samples_split=10,
    min_samples_leaf=5,
    ccp_alpha=best_alpha,
    random_state=42
)
tree_model.fit(X_train, y_train)

# Evaluar el modelo
train_accuracy = accuracy_score(y_train, tree_model.predict(X_train))
test_accuracy = accuracy_score(y_test, tree_model.predict(X_test))

# Imprimir métricas
print(f'Train Accuracy: {train_accuracy:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

# Generar curva de aprendizaje con tamaños reducidos
train_sizes = np.linspace(0.1, 1.0, 6) * len(X_train)  # Reducir a 6 puntos para menor tiempo de cómputo
train_sizes = train_sizes.astype(int)

train_scores = []
test_scores = []

for train_size in train_sizes:
    X_train_subset = X_train.iloc[:train_size]  # Asegurar selección de filas correcta
    y_train_subset = y_train.iloc[:train_size]
    
    # Entrenar solo una vez por cada tamaño
    tree_model.fit(X_train_subset, y_train_subset)
    train_scores.append(accuracy_score(y_train_subset, tree_model.predict(X_train_subset)))
    test_scores.append(accuracy_score(y_test, tree_model.predict(X_test)))

# Graficar curvas de aprendizaje
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_scores, label='Train Accuracy', marker='o')
plt.plot(train_sizes, test_scores, label='Test Accuracy', marker='o')
plt.xlabel('Training Set Size')
plt.ylabel('Accuracy')
plt.title('Learning Curves (Decision Tree) - Optimized')
plt.legend()
plt.show()


---
> He hecho mas ajustes para obtener mejores resultados

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt

# Dividir los datos en conjuntos de entrenamiento y prueba
X = df.drop('HasDetections', axis=1)
y = df['HasDetections']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Obtener valores de poda (ccp_alpha) con muestreo reducido
tree = DecisionTreeClassifier(random_state=42)
path = tree.cost_complexity_pruning_path(X_train, y_train)
ccp_alphas = path.ccp_alphas

# Filtrar valores negativos y reducir la cantidad de valores de ccp_alpha
ccp_alphas = ccp_alphas[ccp_alphas >= 0]
ccp_alphas = np.linspace(min(ccp_alphas), max(ccp_alphas), num=5)  # Seleccionar solo 5 valores para evaluar

# Seleccionar el mejor ccp_alpha con menos cálculos
if len(ccp_alphas) > 0:
    alpha_scores = []

    for alpha in ccp_alphas:
        tree = DecisionTreeClassifier(random_state=42, ccp_alpha=alpha)
        scores = cross_val_score(tree, X_train, y_train, cv=2)  # Reducir a cv=2 para mayor velocidad
        alpha_scores.append(scores.mean())

    best_alpha = ccp_alphas[np.argmax(alpha_scores)]
else:
    best_alpha = 0.0

# Crear y entrenar el modelo optimizado con más profundidad
tree_model = DecisionTreeClassifier(
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=3,
    ccp_alpha=best_alpha,
    random_state=42
)
tree_model.fit(X_train, y_train)

# Evaluar el modelo de árbol de decisión
train_accuracy = accuracy_score(y_train, tree_model.predict(X_train))
test_accuracy = accuracy_score(y_test, tree_model.predict(X_test))

print(f'Decision Tree - Train Accuracy: {train_accuracy:.4f}')
print(f'Decision Tree - Test Accuracy: {test_accuracy:.4f}')

# Entrenar un Random Forest para mejorar la generalización
rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf_model.fit(X_train, y_train)

# Evaluar el modelo de Random Forest
rf_train_accuracy = accuracy_score(y_train, rf_model.predict(X_train))
rf_test_accuracy = accuracy_score(y_test, rf_model.predict(X_test))

print(f'Random Forest - Train Accuracy: {rf_train_accuracy:.4f}')
print(f'Random Forest - Test Accuracy: {rf_test_accuracy:.4f}')

# Generar curva de aprendizaje con tamaños reducidos
train_sizes = np.linspace(0.1, 1.0, 5) * len(X_train)  # Reducir a 5 puntos clave
train_sizes = train_sizes.astype(int)

train_scores = []
test_scores = []

for train_size in train_sizes:
    X_train_subset = X_train.iloc[:train_size]
    y_train_subset = y_train.iloc[:train_size]
    
    tree_model.fit(X_train_subset, y_train_subset)
    train_scores.append(accuracy_score(y_train_subset, tree_model.predict(X_train_subset)))
    test_scores.append(accuracy_score(y_test, tree_model.predict(X_test)))

# Graficar curvas de aprendizaje
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_scores, label='Decision Tree - Train Accuracy', marker='o')
plt.plot(train_sizes, test_scores, label='Decision Tree - Test Accuracy', marker='o')
plt.xlabel('Training Set Size')
plt.ylabel('Accuracy')
plt.title('Learning Curves (Decision Tree) - Improved')
plt.legend()
plt.show()